In [3]:
import pandas as pd
import numpy as np
import copy

In [4]:
#wyczytanie danych
ratings = pd.read_csv('ml-100k/u.data', sep='\t', header=None)
ratings



,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [5]:
#towrzenie tablicy tyle wierszy ile jest użytkowników
R=ratings.pivot(index=0, columns=1, values=2).fillna(0).values #index to co ma być wierszami,nazwa kolumny lub numer,value to wartości wypełniające
R
#fillna(0) - wypełnienie pustych miejsc zerami
#values - zwraca wartości w tabeli ndarray

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [6]:
#tworzennie danych naiwnych aby się szybicej liczyło 0-oznacza brak oceny
A = np.array([[4,5,4,0],[3,4,0,4],[2,0,5,5]])
A

array([[4, 5, 4, 0],
       [3, 4, 0, 4],
       [2, 0, 5, 5]])

In [7]:
#macierze masek czyli boolowa macierz w kształcie maicerzy czy dana ocena jest faktycznie nadan czy nie
maska = A>0
maska

array([[ True,  True,  True, False],
       [ True,  True, False,  True],
       [ True, False,  True,  True]])

In [8]:
no_user=A.shape[0]
no_movie=A.shape[1]
no_l=2 #liczba ukrytych zmiennych

eta=0.01 #współczynnik uczenia

In [9]:
#stworzenie początkowych wartości macierzy 
Q= np.random.rand(no_user,no_l) #Kształt macierzy A liczba wierszy to liczba użytkwoników, liczba kolumn to liczba cech
P= np.random.rand(no_l,no_movie) #Kształt macierzy B liczba wierszy to liczba cech, liczba kolumn to liczba filmów
Q , P

(array([[0.40656639, 0.1400929 ],
        [0.89542416, 0.35632201],
        [0.62336153, 0.22619594]]),
 array([[0.71384442, 0.38704128, 0.36685499, 0.41849586],
        [0.32551569, 0.53706327, 0.18465312, 0.5044533 ]]))

In [10]:
#Uczenie macierzy i tutaj w pdf są wzory na aktualzjacje macierzy p i q zaczniemy od macierzy q czyli użytkowników
for i in range(Q.shape[0]): #iteracja po użytkownikach
    for j in range(Q.shape[1]): #iteracja po cechach
        #musmy przyjąć jakiś błąd na tym etapie zakłądam że P jest ustalone i policzę błąd
        grad = 0
        #ale moge go liczyć po elemntach gdzie wartości są podane
        for idl , l in enumerate(A[i,:]):
            if (l):
                #liczymy tą sume
                grad += A[i,idl] - (Q[i,0]*P[0,idl]+Q[i,1]*P[1,idl])*P[j,idl]
        #teraz aktualizacja
        Q[i,j] = Q[i,j] + eta*grad
        
Q

array([[0.53262679, 0.266787  ],
       [0.99563084, 0.45752495],
       [0.737098  , 0.34153533]])

In [ ]:
# # Iterujemy po wszystkich kolumnach macierzy P (filmy)
# for j in range(P.shape[1]):  # P.shape[1] to liczba filmów
#     # Iterujemy po wszystkich wierszach macierzy P (cechy)
#     for i in range(P.shape[0]):  # P.shape[0] to liczba cech filmów
#         grad = 0  # Inicjalizacja gradientu dla danego elementu macierzy P
#         # Iterujemy po wszystkich użytkownikach (wierszach macierzy A)
#         for idl, l in enumerate(A[:, j]):  # A[:, j] to wszystkie oceny dla filmu `j`
#             if l:  # Sprawdzamy, czy wartość oceny istnieje (nie jest zerem)
#                 # Obliczamy różnicę pomiędzy faktyczną oceną a przewidywaną oceną
#                 # Przewidywana ocena: suma iloczynów cech użytkownika i filmu
#                 pred = Q[idl, 0] * P[0, j] + Q[idl, 1] * P[1, j]
#                 grad += (A[idl, j] - pred) * Q[idl, i]  # Dodanie składnika gradientu
#         # Aktualizujemy element macierzy P na podstawie gradientu
#         # eta to współczynnik uczenia, który kontroluje wielkość kroku
#         P[i, j] = P[i, j] + eta * grad

# # Zwracamy (lub sprawdzamy) macierz P po aktualizacji
# P


In [11]:
#to samo dla maceirzy P czyli filmów

for j in range(P.shape[1]): 
    for i in range(P.shape[0]):
        grad = 0
        for idl , l in enumerate(A[:,j]):
            if (l):
                #liczymy tą sume
                grad += A[idl,j] - (Q[idl,0]*P[0,j]+Q[idl,1]*P[1,j])*Q[idl,i]
        #teraz aktualizacja
        P[i,j] = P[i,j] + eta*grad\
            
P

array([[0.78809986, 0.46889699, 0.45309384, 0.49850579],
       [0.40752903, 0.62275455, 0.27252183, 0.58928312]])

In [12]:
#Możemy sprawdzić w jakim stpopniu przybliża

pred = np.matmul(Q,P)
MAE=np.abs(A - pred).mean()
MAE

2.674479852639403

In [13]:
#Wypadałoby pzeprowadić uczenie kilka razy a nie tylko raz

def train (Q,P, A, eta=0.01, epok=10):
    #chcemy aby miaceirez uczyly się niezależnie od siebie dodajem q1 i p1
    for e in range(epok):
        Q1= np.zeros(Q.shape)
        for i in range(Q.shape[0]): 
            for j in range(Q.shape[1]): 
                grad = 0
                for idl , l in enumerate(A[i,:]):
                    if (l):
                        #grad += (A[i,idl] - (Q[i,0]*P[0,idl]+Q[i,1]*P[1,idl]))*P[j,idl]
                        grad += (A[i,idl] - np.dot(Q[i,:], P[:,idl]))*P[j,idl]
                #uśredniamy gradienty po to aby sieć się uczyła jaka jest dużo ocen unikamy za dużych gradientów
                grad/= sum(A[i,:]>0)
                Q1[i,j] = Q[i,j] + eta*grad
        P1= np.zeros(P.shape)       
        for j in range(P.shape[1]): 
            for i in range(P.shape[0]):
                grad = 0
                for idl , l in enumerate(A[:,j]):
                    if (l):
                        #grad += (A[idl,j] - (Q[idl,0]*P[0,j]+Q[idl,1]*P[1,j]))*Q[idl,i]
                        grad += (A[idl,j] - np.dot(Q[idl,:], P[:,j]))*Q[idl,i]
                #uśredniamy gradienty po to aby sieć się uczyła jaka jest dużo ocen unikamy za dużych gradientów
                grad/= sum(A[:,j]>0)
                P1[i,j] = P[i,j] + eta*grad
                
                
        Q=copy.deepcopy(Q1)
        P=copy.deepcopy(P1)        
        pred = np.matmul(Q,P)
        MAE=np.abs(A - pred).mean()
        print(f'Epoka{e} \t MAE: {MAE}')
    
    return Q,P

In [14]:
Q= np.random.rand(no_user,no_l) 
P= np.random.rand(no_l,no_movie) 


In [15]:
Q , P= train(Q,P,A,0.01, 2000)

Epoka0 	 MAE: 2.6740527133654184
Epoka1 	 MAE: 2.651551601803483
Epoka2 	 MAE: 2.628033241156935
Epoka3 	 MAE: 2.603484650868636
Epoka4 	 MAE: 2.577898184389774
Epoka5 	 MAE: 2.551272112688353
Epoka6 	 MAE: 2.5236111860224786
Epoka7 	 MAE: 2.49492715863418
Epoka8 	 MAE: 2.4652392598300454
Epoka9 	 MAE: 2.434574594162564
Epoka10 	 MAE: 2.4029684532257893
Epoka11 	 MAE: 2.370464522035253
Epoka12 	 MAE: 2.3371149641587783
Epoka13 	 MAE: 2.3029803717550212
Epoka14 	 MAE: 2.268129569473205
Epoka15 	 MAE: 2.232639264735258
Epoka16 	 MAE: 2.196593541172315
Epoka17 	 MAE: 2.160083196778802
Epoka18 	 MAE: 2.1232049334862735
Epoka19 	 MAE: 2.08606041011085
Epoka20 	 MAE: 2.0487551757291054
Epoka21 	 MAE: 2.0113975052127953
Epoka22 	 MAE: 1.974097162636897
Epoka23 	 MAE: 1.9369641213311717
Epoka24 	 MAE: 1.9001072712848714
Epoka25 	 MAE: 1.863633145313414
Epoka26 	 MAE: 1.8276446948065328
Epoka27 	 MAE: 1.7922401440305056
Epoka28 	 MAE: 1.7575119489606685
Epoka29 	 MAE: 1.7235458826503924
Epoka30

In [16]:
pred= np.matmul(Q,P)
pred,A #mogą pojaiwć si wartości spoza zakresu 0-5 i należałoby to ogranizyć

(array([[3.99555843, 5.00464155, 3.9988596 , 4.77089607],
        [3.00562685, 3.99401843, 3.4822463 , 4.00161735],
        [1.99984752, 3.95560155, 5.00095043, 4.99902442]]),
 array([[4, 5, 4, 0],
        [3, 4, 0, 4],
        [2, 0, 5, 5]]))

In [17]:
#NIe wsytarczy tylko podmienić macierz A na R ponieważ napisaliśmy gradienty dla dwueelemeno
no_user=R.shape[0]
no_movie=R.shape[1]
no_l=100
Q= np.random.rand(no_user,no_l) 
P= np.random.rand(no_l,no_movie) 


Q , P= train(Q,P,R,0.01, 4)

KeyboardInterrupt: 

In [18]:
Q,P = train(Q,P,R,0.01, 100)

KeyboardInterrupt: 

In [19]:
#liczenie prwadziwego błędu ponieważ nie uwzględniliśmy maski i liczyliśy na wszystkich a gradenity tylko na wybranych
maska = R>0
pred = np.matmul(Q,P)
MEA= np.abs(R - np.multiply(pred,maska)).mean()
MEA

1.353070983029567

In [20]:
t1= np.array([[1,2,3],[4,5,6]])
m1= np.array([[0,1,0],[1,0,1]])
np.multiply(t1,m1) 

array([[0, 2, 0],
       [4, 0, 6]])